In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from sklearn.metrics import accuracy_score
import numpy as np

In [3]:
train_dir = 'C:/Users/Aziz/Desktop/Alzheimer_s Dataset/Alzheimer_s Dataset/train'  
test_dir = 'C:/Users/Aziz/Desktop/Alzheimer_s Dataset/Alzheimer_s Dataset/test'

In [5]:
img_size = (128, 128)
batch_size = 32

In [7]:
datagen = ImageDataGenerator(rescale=1.0/255.0, validation_split=0.2)

In [11]:
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    subset='training',
    class_mode='categorical'
)

Found 4098 images belonging to 4 classes.


In [13]:
test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    subset='validation',
    class_mode='categorical'
)

Found 254 images belonging to 4 classes.


In [15]:
# Step 2: Build the CNN Feature Extractor
cnn_model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(256, activation='relu'),
])

C:\Users\Aziz\Downloads\Programs\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [17]:
cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [19]:
# Step 3: Extract Features Using CNN
features_train = cnn_model.predict(train_generator)
features_test = cnn_model.predict(test_generator)

C:\Users\Aziz\Downloads\Programs\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


129/129 ━━━━━━━━━━━━━━━━━━━━ 7s 56ms/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step


In [21]:
# Save the CNN model
cnn_model.save('cnn_feature_extractor.h5')

In [23]:
# Step 4: Build and Train the ANN Model
ann_model = Sequential([
    Input(shape=(features_train.shape[1],)),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(4, activation='softmax')  # Adjust the number of classes
])

In [25]:
ann_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [27]:
# Convert labels to one-hot encoding
train_labels = np.array(train_generator.labels)
test_labels = np.array(test_generator.labels)
train_labels_one_hot = tf.keras.utils.to_categorical(train_labels, num_classes=4)
test_labels_one_hot = tf.keras.utils.to_categorical(test_labels, num_classes=4)


In [29]:
# Train the ANN model
ann_model.fit(features_train, train_labels_one_hot, validation_data=(features_test, test_labels_one_hot), epochs=10)


Epoch 1/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.4718 - loss: 1.1504 - val_accuracy: 0.5039 - val_loss: 1.0266
Epoch 2/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5032 - loss: 1.0288 - val_accuracy: 0.5039 - val_loss: 1.0274
Epoch 3/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5101 - loss: 1.0321 - val_accuracy: 0.5039 - val_loss: 1.0274
Epoch 4/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5014 - loss: 1.0466 - val_accuracy: 0.5039 - val_loss: 1.0276
Epoch 5/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5134 - loss: 1.0277 - val_accuracy: 0.5039 - val_loss: 1.0280
Epoch 6/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5012 - loss: 1.0344 - val_accuracy: 0.5039 - val_loss: 1.0279
Epoch 7/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5003 - loss: 1.0481 - val_accuracy: 0.5039 - val_loss: 1.0275
Epoch 8/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4994 - loss: 1.0363 - val_accuracy: 0.

In [31]:
# Calculate accuracy
predictions = np.argmax(ann_model.predict(features_test), axis=1)
test_accuracy = accuracy_score(test_labels, predictions)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Test Accuracy: 50.39%


In [33]:
# Save the ANN model
ann_model.save('ann_model.h5')

In [35]:
# Step 5: Classify New Images
def classify_image(img_path):
    # Load the saved CNN and ANN models
    cnn_model = tf.keras.models.load_model('cnn_feature_extractor.h5')
    ann_model = tf.keras.models.load_model('ann_model.h5')

    # Load and preprocess the new image
    img = image.load_img(img_path, target_size=img_size)  # Resize to match CNN input
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Normalize

    # Extract features using the CNN model
    features = cnn_model.predict(img_array)
    features = features.reshape(1, -1)  # Flatten for ANN

    # Classify the features with the ANN model
    prediction = ann_model.predict(features)
    predicted_class = np.argmax(prediction, axis=1)  # Get the class with the highest probability

    # Interpret the prediction
    class_labels = {0: 'MildDemented', 1: 'ModerateDemented', 2: 'NonDemented', 3: 'VeryMildDemented'}
    result = class_labels[predicted_class[0]]
    return result

# Test the function with a new image
img_path = r'C:/Users/Aziz/Desktop/Alzheimer_s Dataset/Alzheimer_s Dataset/test/NonDemented/27 (89).jpg'
result = classify_image(img_path)
print(f"The image is classified as: {result}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 186ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
The image is classified as: NonDemented
